In [28]:
import os
import openai
import sys

env = {}

# read .env file and set variables
with open('.env') as f:
    for line in f:
        key, value = line.strip().split('=')
        env[key] = value

openai.api_key = env['OPEN_AI_KEY']
OPENAI_API_KEY = env['OPEN_AI_KEY']

In [2]:
pip install "unstructured[md]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 9.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [3]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader


In [4]:

# Load md's
loaders = [
    UnstructuredMarkdownLoader('testdocs/product_eclipse_x1.md'),
    UnstructuredMarkdownLoader('testdocs/product_washing_machine.md'),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

[nltk_data] Downloading package punkt to /Users/pikkuaapo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/pikkuaapo/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [6]:
splits = text_splitter.split_documents(docs)

In [7]:
from langchain.vectorstores import Chroma

In [8]:
persist_directory = 'testdocs/chroma/'

In [30]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=env['OPEN_AI_KEY'])

In [31]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [32]:
print(vectordb._collection.count())

16


In [33]:
question = "How many cores does X1 have?"

In [34]:
docs = vectordb.similarity_search(question,k=5)

In [35]:
docs[0]

Document(metadata={'source': 'testdocs/product_eclipse_x1.md'}, page_content='Efficiency Cores (EC):\n\nCore Count: 16 efficiency cores\n\nClock Speed: 2.5 GHz\n\nArchitecture: Energy-efficient ARM v10 design optimized for low-power operations, background tasks, and always-on functionalities.\n\nNeural Processing Unit (NPU):\n\nCore Count: 4 specialized AI cores\n\nCapabilities: Optimized for machine learning, neural network inference, and AI-driven applications, with support for INT8, FP16, and BFLOAT16 data types.\n\nGraphics Processing Unit (GPU):\n\nArchitecture: Custom 2048-core GPU based on AMD RDNA 4 architecture\n\nFeatures: Ray tracing, AI-driven image enhancement, and support for 8K resolution displays.\n\nPerformance Metrics\n\nThe Eclipse X1 processor delivers exceptional performance across a range of benchmarks, reflecting its capability to handle diverse workloads efficiently:\n\nSingle-Core Performance:\n\nThe high-performance cores achieve industry-leading single-thread

In [37]:
from langchain.chat_models import ChatOpenAI
llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=env['OPEN_AI_KEY'])

In [38]:
from langchain.chains import RetrievalQA

In [39]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [40]:
result = qa_chain({"query": question})

In [41]:
result["result"]

'The Eclipse X1 processor has a total of 24 cores, consisting of 8 high-performance cores and 16 efficiency cores.'

In [42]:
question = "Can I use voice commands with my AI-powered washing machine?"

In [43]:
result = qa_chain({"query": question})

In [44]:
result["result"]

'Yes, you can use voice commands with your AI-powered washing machine. You can start, stop, and monitor wash cycles using voice assistants like Alexa and Google Assistant.'

In [45]:
question = "What are the distuingishing features of my AI-powered washing machine compared to typical washing machines?"

In [47]:
result = qa_chain({"query": question})

In [48]:
result["result"]

'The distinguishing features of your AI-powered washing machine compared to typical washing machines include:\n\n1. Intelligent Load Sensing: Automatically detects fabric type, load size, and soil level for optimal wash cycles.\n2. AI-Powered Stain Detection: Identifies and treats different types of stains with precision.\n3. Voice Control & Smart Integration: Compatible with smart home systems and voice assistants for seamless operation.\n4. Eco-Friendly Washing: Minimizes water and energy consumption without compromising performance.\n5. Optimized Water Usage: Uses just the right amount of water for each load, reducing waste and saving on utility bills.\n6. Enhanced User Experience: Enjoy a seamless, connected home environment with customizable alerts and remote control features.'